Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [135]:

import copy
from dataclasses import dataclass
from random import choice, randint
import random
import lab9_lib


MUTATION_PROBABILITY = 0.15
TOURNAMENT_SIZE = 2
GEN_SIZE = 1000
PROB_INSTANCE = 2



In [133]:
@dataclass
class Individual:
    fitness: float
    genotype: list[bool]

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = copy.copy(ind)
    pos = randint(0, GEN_SIZE-1)
    offspring.genotype[pos] = 1 -  offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, GEN_SIZE-1)
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    assert len(offspring.genotype) == GEN_SIZE
    return offspring

### GENERAL FUNCTIONS, USEFUL FOR ALL THE APPROACHES


In [134]:
# GENERAL FUNCTIONS, USEFUL FOR ALL THE APPROACHES


def binary_to_string(genome):
    return ''.join(str(g) for g in genome)

def fitness_f(genome):
    return lab9_lib.make_problem(PROB_INSTANCE)(genome)

def local_search(population,pop_size,off_size,fitness_counter):
    offspring = list()
    for _ in range(off_size):
        if random.random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # mutation  # add more clever mutations
            p = select_parent(population)
            o = mutate(p)
        else:
            # xover # add more xovers
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = one_cut_xover(p1, p2)
        offspring.append(o)

    for i in offspring:
        i.fitness = fitness_f(i.genotype)
        fitness_counter+=1
    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:pop_size]
    return fitness_counter
    #print("Best fitness in local_search: ",population[0].fitness)



# ISLAND APPROACH


In [93]:


from copy import deepcopy
import random
import lab9_lib
from tqdm.notebook import tqdm


class Island:

    def __init__(self, population_size,off_size,max_iter):
        self.population = []
        self.best_individual = None
        self.best_fitness = None
        self.population_size = population_size
        self.max_iterations = max_iter
        self.off_size = off_size

    def gen_population(self, population : list):
        new_population = []
        for _ in range(self.population_size):
            individual = random.choice(population)
            individual.fitness = fitness_f(individual.genotype)
            new_population.append(individual)
            population.remove(individual)       #updated variable -> the picked individuals have been removed
        self.population = new_population
        
    
    def evolve(self,fitness_counter):
        for _ in range(self.max_iterations):
            fitness_counter = local_search(self.population,self.population_size,self.off_size,fitness_counter)
        self.best_fitness =  self.get_best_fitness()
        #print("Island best fitness: ",self.best_fitness)
        self.best_individual =  self.get_best_individual()
        #print("Island best ind: ",self.best_individual)
        return fitness_counter

    def get_best_fitness(self):
        if self.best_fitness is None:
            self.best_fitness = float("-inf")
        for individual in self.population:
            fit = fitness_f(individual.genotype)
            if fit > self.best_fitness:
                self.best_fitness = fit
                self.best_individual = individual
        return self.best_fitness

    def get_best_individual(self):
        if self.best_individual is None:
            self.best_fitness = float("-inf")
        for individual in self.population:
            fit = fitness_f(individual.genotype)
            if fit > self.best_fitness:
                self.best_fitness = fit
                self.best_individual = individual
        return self.best_individual


    def __repr__(self):
        return f"Island(population={self.population}, best_individual={self.best_individual}, best_fitness={self.best_fitness})"
    
             ####################### end of class Island implementation #######################



def island_model(population,population_size, island_size, max_iterations,island_max_iter):
    best_fitness = 0.0
    best_genome = None
    fitness_counter = 0

    island_count = population_size // island_size
    
    islands = []
    pop_cpy = deepcopy(population)
    history = list()

    #generation and population of islands
    for _ in range(island_count):
        island = Island(population_size=island_size,off_size=island_size*10 ,max_iter=island_max_iter)
        island.gen_population(pop_cpy)
        islands.append(island)

    for c in tqdm(range(max_iterations)):
        #each island evolves on its own, indipendently -> each iteration here represents a new epoch for the population inside the islands
        for island in islands:
            fitness_counter = island.evolve(fitness_counter)
            if island.best_individual.fitness > best_fitness:
                best_fitness = island.best_individual
            
        history.append((c,best_fitness))

        if (c%(max_iterations/10) == 0):      #each max_iterations/10 iterations apply migration
            # exchange the best individuals according to fitness -> migration concept applied here -> used to improve fitness and bring new info from another search space area
            for i in range(island_count // 2):
                islands[i].best_individual, islands[island_count - 1 - i].best_individual = islands[island_count - 1 - i].best_individual, islands[i].best_individual


       

        #find the best fitness among the several islands
        for island in islands:
            #print(island)
            if island.get_best_individual().fitness > best_fitness:
                best_fitness = island.best_individual.fitness
                best_genome = binary_to_string(island.best_individual.genotype)
                best_ind = island.best_individual

    return best_fitness, best_genome,best_ind,fitness_counter,history






# CELLULAR ES APPROACH


In [136]:
#CELLULAR EA
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

#fitness class -> useful to take the count of the fitness calls
class FitnessFunction:
    def __init__(self,problem_instance):
        self.counter = 0
        self.fit = lab9_lib.make_problem(problem_instance)
    def fit_funct(self,individual):
        self.counter +=1
        return self.fit(individual)


def create_population(rows, cols, individual_length):
    return np.random.randint(2, size=(rows, cols, individual_length))

# parent selection
def selection(population, fitnesses, x, y):
    neighbors = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]
    neighbors = [(i%population.shape[0], j%population.shape[1]) for i, j in neighbors]
    neighbor_fitnesses = [fitnesses[i, j] for i, j in neighbors]     #the lattice forces a fixed structure -> only local interactions are allowed so you can only take the neighbors
    return population[neighbors[np.argmax(neighbor_fitnesses)]]


# mutation or xover are applied
def mutation_or_xover(individual, mutation_rate,parent1, parent2):
        if np.random.random() < mutation_rate:
            offspring = copy.copy(individual)
            pos = randint(0, GEN_SIZE-1)
            offspring[pos] = 1 -  offspring[pos]
        else:
            crossover_index = np.random.randint(len(parent1))
            offspring = np.concatenate((parent1[:crossover_index], parent2[crossover_index:]))
        return offspring

# the algorithm is run here -> look at how the double concatenated for force a 2D lattice
def cellular_genetic_algorithm(population, fitness, selection, mutation_or_xover, mutation_rate, generations,elitism_rate):
    fitnesses = np.apply_along_axis(fitness, 2, population)
    history = list()
    best_fitness = None
    best_genome = None

  

    for c in range(generations):
        print("iteration: ",c)
        # Preserva i migliori individui della popolazione
        best_individuals = population[fitnesses.argsort()[:int(population.shape[0] * elitism_rate)]]
        # Seleziona i genitori per la riproduzione
        for i in range(population.shape[0]):
            for j in range(population.shape[1]):
                parent1 = population[i, j]
                parent2 = selection(population, fitnesses, i, j)

                # Applica la mutazione o l'incrocio
                child = mutation_or_xover(parent1, mutation_rate, parent1, parent2)

                # Sostituisci l'individuo peggiore con il figlio
                if fitnesses[i, j] < fitnesses[best_individuals.shape[0] - 1, 0]:
                    population[i, j] = child
                    best_individuals[best_individuals.shape[0] - 1] = child

                if best_fitness == None or fitnesses[i,j] > best_fitness:
                  best_fitness = fitnesses[i,j]
                  best_genome = population[i,j]

        # Aggiorna i fitness degli individui
        fitnesses = np.apply_along_axis(fitness, 2, population)

        # Aggiorna la storia
        history.append((c, best_fitness))

    return best_fitness, best_genome, history


#main below
rows = 100
cols = 100
individual_length = 1000
mutation_rate = 0.15
generations = 100
problem_instance = 10
elitism_rate  = 15

fitness_function = FitnessFunction(problem_instance)

#generation of the population
population = create_population(rows, cols, individual_length)

#execution of the algorithm
best_fitness,best_genome,history = cellular_genetic_algorithm(population, fitness_function.fit_funct, selection, mutation_or_xover, mutation_rate, generations,elitism_rate)


print('Miglior individuo:', best_genome)
print("Migliore fitness: ",best_fitness)
print("Numero di fitness call: ",fitness_function.counter)

history = np.array(history)
plt.figure(figsize=(14, 4))
plt.plot(history[:, 0], history[:, 1], marker=".")


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "d:\Desktop\II ANNO LM\Computational Intelligence\CI - Exercises\CI - Prof_folder\computational-intelligence\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Marco\AppData\Local\Temp\ipykernel_16660\2140756604.py", line 92, in <module>
    best_fitness,best_genome,history = cellular_genetic_algorithm(population, fitness_function.fit_funct, selection, mutation_or_xover, mutation_rate, generations,elitism_rate)
                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Marco\AppData\Local\Temp\ipykernel_16660\2140756604.py", line -1, in cellular_genetic_algorithm
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "d:\Desktop\II ANNO L

In [95]:
import numpy as np
import matplotlib.pyplot as plt
# define the input parameters
population_size = 50
island_size = 10
max_iterations = 15
island_max_iter = 15

# generate the initial population
population = [
    Individual(
        genotype=[choice((0, 0)) for _ in range(GEN_SIZE)],
        fitness=None,
    )
    for _ in range(population_size)
]

# run the island model
best_fitness, best_genome, best_ind,fitness_counter,history = island_model(population, population_size, island_size, max_iterations, island_max_iter)

# print the results
print("Best fitness:", best_fitness)
print("Best genome:", best_genome)
print("Best individual: ",best_ind)
print("Fitness calls: ",fitness_counter)

history = np.array(history)
plt.figure(figsize=(14, 4))
plt.plot(history[:, 0], history[:, 1], marker=".")


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [ ]:

import copy
from dataclasses import dataclass
from random import choice, randint
import random


import lab9_lib

 # population initialized here
μ = 50  #population size
λ = 200 #offspring size
σ = 0.001   
MUTATION_PROBABILITY = 0.5
TOURNAMENT_SIZE = 2
GEN_SIZE = 1000



In [ ]:
## you should try with all the possibile optimisations in order to creare a population with a fitness as much closer to 100% as possible


#the genome is fixed (0 and 1 sequence)

#define the mutation

#define the parent selection -> apply the optimisations

#define the survival selection -> apply the optimisations



In [ ]:
@dataclass
class Individual:
    fitness: float
    genotype: list[bool]

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = copy.copy(ind)
    pos = randint(0, GEN_SIZE-1)
    offspring.genotype[pos] = 1 -  offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, GEN_SIZE-1)
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    assert len(offspring.genotype) == GEN_SIZE
    return offspring

In [ ]:
# GENERAL FUNCTIONS, USEFUL FOR ALL THE APPROACHES
PROB_INSTANCE = 10

def binary_to_string(genome):
    return ''.join(str(g) for g in genome)

def fitness_f(genome):
    return lab9_lib.make_problem(PROB_INSTANCE)(genome)

def local_search(population,pop_size,off_size,fitness_counter):
    offspring = list()
    for _ in range(off_size):
        if random.random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # mutation  # add more clever mutations
            p = select_parent(population)
            o = mutate(p)
        else:
            # xover # add more xovers
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = one_cut_xover(p1, p2)
        offspring.append(o)

    for i in offspring:
        i.fitness = fitness_f(i.genotype)
        fitness_counter+=1
    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:pop_size]
    return fitness_counter
    #print("Best fitness in local_search: ",population[0].fitness)



In [ ]:
from copy import deepcopy
import random
import lab9_lib
from tqdm.notebook import tqdm


class Island:

    def __init__(self, population_size,off_size,max_iter):
        self.population = []
        self.best_individual = None
        self.best_fitness = None
        self.population_size = population_size
        self.max_iterations = max_iter
        self.off_size = off_size

    def gen_population(self, population : list):
        new_population = []
        for _ in range(self.population_size):
            individual = random.choice(population)
            individual.fitness = fitness_f(individual.genotype)
            new_population.append(individual)
            population.remove(individual)       #updated variable -> the picked individuals have been removed
        self.population = new_population
        
    
    def evolve(self,fitness_counter):
        for _ in range(self.max_iterations):
            fitness_counter = local_search(self.population,self.population_size,self.off_size,fitness_counter)
        self.best_fitness =  self.get_best_fitness()
        #print("Island best fitness: ",self.best_fitness)
        self.best_individual =  self.get_best_individual()
        #print("Island best ind: ",self.best_individual)
        return fitness_counter

    def get_best_fitness(self):
        if self.best_fitness is None:
            self.best_fitness = float("-inf")
        for individual in self.population:
            fit = fitness_f(individual.genotype)
            if fit > self.best_fitness:
                self.best_fitness = fit
                self.best_individual = individual
        return self.best_fitness

    def get_best_individual(self):
        if self.best_individual is None:
            self.best_fitness = float("-inf")
        for individual in self.population:
            fit = fitness_f(individual.genotype)
            if fit > self.best_fitness:
                self.best_fitness = fit
                self.best_individual = individual
        return self.best_individual


    def __repr__(self):
        return f"Island(population={self.population}, best_individual={self.best_individual}, best_fitness={self.best_fitness})"
    
             ####################### end of class Island implementation #######################



def island_model(population,population_size, island_size, max_iterations,island_max_iter):
    best_fitness = 0.0
    best_genome = None
    fitness_counter = 0

    island_count = population_size // island_size
    
    islands = []
    pop_cpy = deepcopy(population)
    history = list()

    #generation and population of islands
    for _ in range(island_count):
        island = Island(population_size=island_size,off_size=island_size*10 ,max_iter=island_max_iter)
        island.gen_population(pop_cpy)
        islands.append(island)

    for c in tqdm(range(max_iterations)):
        #each island evolves on its own, indipendently -> each iteration here represents a new epoch for the population inside the islands
        for island in islands:
            fitness_counter = island.evolve(fitness_counter)
            if island.best_individual.fitness > best_fitness:
                best_fitness = island.best_individual
            
        history.append((c,best_fitness))

        if (c%(max_iterations/10) == 0):      #each max_iterations/10 iterations apply migration
            # exchange the best individuals according to fitness -> migration concept applied here -> used to improve fitness and bring new info from another search space area
            for i in range(island_count // 2):
                islands[i].best_individual, islands[island_count - 1 - i].best_individual = islands[island_count - 1 - i].best_individual, islands[i].best_individual


       

        #find the best fitness among the several islands
        for island in islands:
            #print(island)
            if island.get_best_individual().fitness > best_fitness:
                best_fitness = island.best_individual.fitness
                best_genome = binary_to_string(island.best_individual.genotype)
                best_ind = island.best_individual

    return best_fitness, best_genome,best_ind,fitness_counter,history






In [ ]:
# import random


# class Cell:
#     def __init__(self, genome):
#         self.genome = genome
#         self.fitness = None

#     def __repr__(self):
#         return f"Cell(genome={self.genome}, fitness={self.fitness})"


# class CellularEA:
#     def __init__(self,population_size, neighborhood_size, off_size):
#         self.population_size = population_size
#         self.neighborhood_size = neighborhood_size
#         self.off_size = off_size

#         self.population = []
#         for _ in range(self.population_size):
#             self.population.append(Cell(random.choices([0, 1],k=population_size)))

#     def evaluate(self):
#         local_search(self.population,self.population_size,off_size=self.off_size)

#     def select(self):
#         # Select the best cell in the neighborhood
#         best_cell = min(self.population, key=lambda cell: cell.fitness)

#         # Create a new cell by mutating the best cell
#         new_cell = Cell(best_cell.genome)
#         random.shuffle(new_cell.genome)
#         for i in range(len(new_cell.genome)):
#             if random.random() < self.mutation_rate:
#                 new_cell.genome[i] = not new_cell.genome[i]

#         return new_cell

#     def iterate(self):
#         self.evaluate()

#         # Replace the worst cell in the population with a new cell
#         worst_cell = max(self.population, key=lambda cell: cell.fitness)
#         wc_index = self.population.index(worst_cell)
#         self.population[wc_index] = self.select()

#     def run(self, iterations):
#         for _ in range(iterations):
#             self.iterate()

#         return self.population[0]


# def main():
#     # define the input parameters
#     population_size = 100
#     max_iterations = 100
#     ea = CellularEA(population_size, 4,population_size*2)
#     best_genome = ea.run(max_iterations)
#     print(best_genome.genome)


# if __name__ == "__main__":
#     main()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# define the input parameters
population_size = 50
island_size = 10
max_iterations = 15
island_max_iter = 15

# generate the initial population
population = [
    Individual(
        genotype=[choice((0, 0)) for _ in range(GEN_SIZE)],
        fitness=None,
    )
    for _ in range(population_size)
]

# run the island model
best_fitness, best_genome, best_ind,fitness_counter,history = island_model(population, population_size, island_size, max_iterations, island_max_iter)

# print the results
print("Best fitness:", best_fitness)
print("Best genome:", best_genome)
print("Best individual: ",best_ind)
print("Fitness calls: ",fitness_counter)

history = np.array(history)
plt.figure(figsize=(14, 4))
plt.plot(history[:, 0], history[:, 1], marker=".")


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
# fitness = lab9_lib.make_problem(10)
# for n in range(10):
#     ind = choices([0, 1], k=50)
#     print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

# print(fitness.calls)


In [ ]:
# fitness = lab9_lib.make_problem(10)
# for n in range(10):
#     ind = choices([0, 1], k=50)
#     print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

# print(fitness.calls)
